## Початкове завантаження бібліотек та набору даних

In [1]:
import pandas as pd
import numpy as np

In [2]:
covid_data = pd.read_csv("covid-variants.csv")

In [3]:
covid_data

,location,date,variant,num_sequences,perc_sequences,num_sequences_total
0,Angola,2020-07-06,Alpha,0,0.0,3
1,Angola,2020-07-06,B.1.1.277,0,0.0,3
2,Angola,2020-07-06,B.1.1.302,0,0.0,3
3,Angola,2020-07-06,B.1.1.519,0,0.0,3
4,Angola,2020-07-06,B.1.160,0,0.0,3
...,...,...,...,...,...,...
100411,Zimbabwe,2021-11-01,Omicron,0,0.0,6
100412,Zimbabwe,2021-11-01,S:677H.Robin1,0,0.0,6
100413,Zimbabwe,2021-11-01,S:677P.Pelican,0,0.0,6
100414,Zimbabwe,2021-11-01,others,0,0.0,6


## Застосування підсумкових функцій

In [4]:
# Вивід загального опису параметрів стовпця "variant" (штампів вірусу)
covid_data.perc_sequences.describe()

count    100416.000000
mean          6.154355
std          21.898989
min          -0.010000
25%           0.000000
50%           0.000000
75%           0.000000
max         100.000000
Name: perc_sequences, dtype: float64

In [5]:
# Вивід середнього значення (загальної кількості cеквенування ДНК для цієї країни на конкретну дату) по стовпцю "num_sequences_total"
covid_data.num_sequences_total.mean()

1509.5824569789675

In [6]:
# Вивід унікальних значень (певного штаму вірусу) по стовпцю "variant", тип object
covid_data.variant.unique()

array(['Alpha', 'B.1.1.277', 'B.1.1.302', 'B.1.1.519', 'B.1.160',
       'B.1.177', 'B.1.221', 'B.1.258', 'B.1.367', 'B.1.620', 'Beta',
       'Delta', 'Epsilon', 'Eta', 'Gamma', 'Iota', 'Kappa', 'Lambda',
       'Mu', 'Omicron', 'S:677H.Robin1', 'S:677P.Pelican', 'others',
       'non_who'], dtype=object)

In [7]:
# Вивід унікальних значень (країн) і як часто вони трапляються по стовпцю "location"
covid_data.location.value_counts()

Bangladesh        1080
Belgium           1080
United States     1080
United Kingdom    1080
France            1080
                  ... 
Montenegro         384
Monaco             360
Fiji               336
Benin              336
Brunei             240
Name: location, Length: 121, dtype: int64

In [8]:
# Вивід відсоткового відхилення загальної кількості cеквенування ДНК для цієї країни на конкретну дату від середнього
num_sequences_total_mean = covid_data.num_sequences_total.mean()
covid_data.num_sequences_total.map(lambda n: ((n - num_sequences_total_mean) / num_sequences_total_mean) * 100)

0        -99.801270
1        -99.801270
2        -99.801270
3        -99.801270
4        -99.801270
            ...    
100411   -99.602539
100412   -99.602539
100413   -99.602539
100414   -99.602539
100415   -99.602539
Name: num_sequences_total, Length: 100416, dtype: float64

In [9]:
# Визначаємо межі третин загальної кількості cеквенування ДНК для цієї країни на конкретну дату
q1 = covid_data.num_sequences_total.quantile(0.33) # Визначаємо межу для нижньої чверті значень, тобто межу 33%
q2 = covid_data.num_sequences_total.quantile(0.66) # Визначаємо межу для верхньої чверті значень, тобто межу 66%
# Вивід загальної кількості cеквенування ДНК для цієї країни на конкретну дату у вигляді категорій
covid_data.num_sequences_total.map(lambda n: 'High' if n > q2 else 'Medium' if n > q1 else 'Low')

0         Low
1         Low
2         Low
3         Low
4         Low
         ... 
100411    Low
100412    Low
100413    Low
100414    Low
100415    Low
Name: num_sequences_total, Length: 100416, dtype: object

In [10]:
# Вивід унікальних значень загальної кількості cеквенування ДНК для цієї країни на конкретну дату у вигляді категорій і як часто вони трапляються
covid_data.num_sequences_total.map(lambda n: 'High' if n > q2 else 'Medium' if n > q1 else 'Low').value_counts()

High      34080
Low       33696
Medium    32640
Name: num_sequences_total, dtype: int64

In [11]:
def remean_sequences(row):
    # Вивід загальної кількості cеквенування ДНК для цієї країни на конкретну дату де за нульове значення вибрано середнє значення
    row.num_sequences_total = row.num_sequences_total - num_sequences_total_mean 
    # Вивід чи відсоток оброблених cеквенування ДНК від загальної кількості cеквенування перевищує 50%
    if row.perc_sequences > 50:
        row.perc_sequences ='The percentage of processed sequences is more than half'
    else:
        row.perc_sequences ='The percentage of processed sequences is less than half'
    return row

covid_data.apply(remean_sequences, axis='columns')

,location,date,variant,num_sequences,perc_sequences,num_sequences_total
0,Angola,2020-07-06,Alpha,0,The percentage of processed sequences is less ...,-1506.582457
1,Angola,2020-07-06,B.1.1.277,0,The percentage of processed sequences is less ...,-1506.582457
2,Angola,2020-07-06,B.1.1.302,0,The percentage of processed sequences is less ...,-1506.582457
3,Angola,2020-07-06,B.1.1.519,0,The percentage of processed sequences is less ...,-1506.582457
4,Angola,2020-07-06,B.1.160,0,The percentage of processed sequences is less ...,-1506.582457
...,...,...,...,...,...,...
100411,Zimbabwe,2021-11-01,Omicron,0,The percentage of processed sequences is less ...,-1503.582457
100412,Zimbabwe,2021-11-01,S:677H.Robin1,0,The percentage of processed sequences is less ...,-1503.582457
100413,Zimbabwe,2021-11-01,S:677P.Pelican,0,The percentage of processed sequences is less ...,-1503.582457
100414,Zimbabwe,2021-11-01,others,0,The percentage of processed sequences is less ...,-1503.582457


In [12]:
# Вивід загальної кількості cеквенування ДНК для цієї країни на конкретну дату де за нульове значення вибрано середнє значення через вбудовані операції над стовпцями
covid_data.num_sequences_total - num_sequences_total_mean

0        -1506.582457
1        -1506.582457
2        -1506.582457
3        -1506.582457
4        -1506.582457
             ...     
100411   -1503.582457
100412   -1503.582457
100413   -1503.582457
100414   -1503.582457
100415   -1503.582457
Name: num_sequences_total, Length: 100416, dtype: float64

In [13]:
# Вивід відношення локації до відсотку оброблених cеквенування ДНК від загальної кількості cеквенування
covid_data.location + " - " + covid_data.variant

0                    Angola - Alpha
1                Angola - B.1.1.277
2                Angola - B.1.1.302
3                Angola - B.1.1.519
4                  Angola - B.1.160
                    ...            
100411           Zimbabwe - Omicron
100412     Zimbabwe - S:677H.Robin1
100413    Zimbabwe - S:677P.Pelican
100414            Zimbabwe - others
100415           Zimbabwe - non_who
Length: 100416, dtype: object